In [ ]:
%pip install lightrag-hku

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 2.6 MB/s eta 0:00:00


In [1]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
import os
import inspect
import logging
from lightrag import LightRAG, QueryParam
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
# from lightrag.llm.hf import hf_model_complete, hf_embed
from lightrag.llm.openai import gpt_4o_mini_complete
from lightrag.utils import EmbeddingFunc
from lightrag.kg.shared_storage import initialize_pipeline_status

from dotenv import load_dotenv

load_dotenv()

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)
WORKING_DIR = "../data/parse/markdown"

async def initialize_rag():
    rag = LightRAG(
        working_dir=WORKING_DIR,
        llm_model_func=gpt_4o_mini_complete,
        
        embedding_func=EmbeddingFunc(
            embedding_dim=1024,
            max_token_size=8192,
            func=lambda texts: ollama_embed(
                texts, embed_model="bge-m3"
            ),
        ),
    )

    await rag.initialize_storages()
    await initialize_pipeline_status()

    return rag


async def print_stream(stream):
    async for chunk in stream:
        print(chunk, end="", flush=True)



In [2]:
def init_rag():
    rag = asyncio.run(initialize_rag())

    with open("../data/parse/markdown/QC tuyển sinh ĐH 2023_Final_1952023.md", "r", encoding="utf-8") as f:
        rag.insert(f.read())
    
    return rag


In [3]:
rag = init_rag()

INFO: Process 33832 Shared-Data created for Single Process
g:\anaconda\anaconda3\envs\chatbot-agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': '../data/parse/markdown\\vdb_entities.json'} 0 data
INFO:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': '../data/parse/markdown\\vdb_relationships.json'} 0 data
INFO:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': '../data/parse/markdown\\vdb_chunks.json'} 0 data
INFO: Process 33832 initialized updated flags for namespace: [full_docs]
INFO: Process 33832 ready to initialize storage namespace: [full_docs]
INFO: Process 33832 initialized updated flags for namespace: [text_chunks]
INFO: Process 33832 ready to initialize storage namespace: [text_chunks]
INFO: Process 33

In [4]:
query = "Những thí sinh được dự tuyển xét tuyển tài năng"
def main(rag):

    # Test different query modes
    print("\nNaive Search:")
    print(
        rag.query(
           query, param=QueryParam(mode="naive")
        )
    )

    print("\nLocal Search:")
    print(
        rag.query(
           query, param=QueryParam(mode="local")
        )
    )

    print("\nGlobal Search:")
    print(
        rag.query(
           query, param=QueryParam(mode="global")
        )
    )

    print("\nHybrid Search:")
    print(
        rag.query(
           query, param=QueryParam(mode="hybrid")
        )
    )

    # stream response
    resp = rag.query(
       query,
        param=QueryParam(mode="hybrid", stream=True),
    )

    if inspect.isasyncgen(resp):
        asyncio.run(print_stream(resp))
    else:
        print(resp)


In [5]:
main(rag)


Naive Search:
### Đối tượng dự tuyển xét tuyển tài năng

Theo Quy chế Tuyển sinh của Đại học Bách khoa Hà Nội, thí sinh được dự tuyển xét tuyển tài năng cần đáp ứng các yêu cầu sau:

1. **Hồ sơ năng lực**: Thí sinh phải nộp hồ sơ đăng ký xét tuyển (ĐKXT) vào ĐHBK Hà Nội theo phương thức xét tuyển tài năng, trong đó bao gồm các minh chứng về kết quả và thành tích học tập, nghiên cứu khoa học, trình độ ngoại ngữ, cũng như các thành tích khác trong các lĩnh vực như văn hóa, nghệ thuật, thể dục thể thao và hoạt động xã hội.

2. **Điều kiện đăng ký**: Thí sinh phải đáp ứng các điều kiện đăng ký dự tuyển mà ĐHBK Hà Nội quy định, trong đó có khả năng về chuyên môn, năng lực học tập và các tiêu chí khác liên quan đến xét tuyển.

### Kết luận

Quy chế Tuyển sinh này giúp tăng cường sự công bằng và công khai trong quá trình tuyển sinh, tạo cơ hội cho những thí sinh có năng lực nổi bật nhưng không chỉ dựa vào kết quả thi cử truyền thống.

### References
- [DC] unknown_source
- [DC] unknown_sourc

In [ ]:
# main(rag)


Naive Search:
## Những Thí Sinh Được Dự Tuyển Xét Tuyển Tài Năng

Theo quy định trong Quy chế Tuyển sinh Đại học Bách khoa Hà Nội, những thí sinh đủ điều kiện dự tuyển xét tuyển tài năng phải đáp ứng một hoặc nhiều tiêu chí sau:

1. **Điểm trung bình học tập**: Thí sinh phải có điểm trung bình chung học tập từng năm học trong chương trình trung học phổ thông đạt từ 8,0 trở lên.

2. **Tham gia các đội tuyển**: Thí sinh tham gia đội tuyển cấp tỉnh hoặc tương đương tham dự kỳ thi chọn học sinh giỏi (HSG) quốc gia trong một trong các môn Toán, Vật lý, Hóa học, Sinh học, Tin học, Ngoại ngữ.

3. **Giải thưởng HSG**: Thí sinh đạt giải Nhất, Nhì, Ba, hoặc Khuyến khích trong kỳ thi chọn HSG cấp tỉnh/thành phố hoặc được chọn tham dự kỳ thi HSG Quốc gia.

4. **Cuộc thi KHKT**: Thí sinh đoạt giải Nhất, Nhì, Ba trong cuộc thi KHKT do Bộ Giáo dục và Đào tạo tổ chức.

5. **Chương trình học chuyên**: Thí sinh học chương trình THPT toàn thời gian theo hệ chuyên ở các trường THPT chuyên hoặc thuộc các 

In [6]:
import pipmaster as pm

if not pm.is_installed("pyvis"):
    pm.install("pyvis")
if not pm.is_installed("networkx"):
    pm.install("networkx")

import networkx as nx
from pyvis.network import Network
import random

# Load the GraphML file
G = nx.read_graphml("../data/parse/text/graph_chunk_entity_relation.graphml")

# Create a Pyvis network
net = Network(height="100vh", notebook=True)

# Convert NetworkX graph to Pyvis network
net.from_nx(G)


# Add colors and title to nodes
for node in net.nodes:
    node["color"] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    if "description" in node:
        node["title"] = node["description"]

# Add title to edges
for edge in net.edges:
    if "description" in edge:
        edge["title"] = edge["description"]

# Save and display the network
net.show("knowledge_graph_md.html")

knowledge_graph_md.html
